In [1]:
import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
data = pd.read_csv("data/interim/preprocessed.tsv", sep='\t', index_col=0)

In [3]:
data.head()

,en,ru
0,"cordelia hotel is situated in tbilisi , a 3min...","отель cordelia расположен в тбилиси , в 3 мину..."
1,at tupirmarka lodge you will find a 24hour fro...,в числе удобств лоджа tupirmarka круглосуточна...
2,"featuring free wifi in all areas , naigao xiao...",апартаменты naigao xiaowo расположены в городе...
3,each has a tv and a private bathroom with show...,в вашем распоряжении также телевизор и собстве...
4,your room comes with air conditioning and sate...,номер оснащен кондиционером и спутниковым теле...


# Prepare Dataloader

In [4]:
from data.make_dataset import prepareData, get_dataloader, tensorFromSentence

# Define model

In [5]:
from translate.model import Transformer, Encoder, Decoder

# Train model

In [6]:
from translate.train import train

In [7]:
epochs = 20
embed_size = 256
n_layer = 6
n_head = 8
hidden_size = 256
batch_size = 32
lr = 5e-5
MAX_LENGTH = 32
model_path = "translate.pt"
start_epoch = 1

vocab_en, vocab_ru, pairs = prepareData(data, MAX_LENGTH)
train_dataloader, val_dataloader = get_dataloader(batch_size, vocab_en, vocab_ru, pairs, MAX_LENGTH, device=device)

Counted words:
en-vocab 29536
ru-vocab 40720


In [8]:
encoder = Encoder(vocab_en.n_words, n_layer, n_head, batch_size, embed_size, hidden_size, vocab_en, device=device, max_length=MAX_LENGTH)
decoder = Decoder(n_layer, n_head, batch_size, embed_size, hidden_size, vocab_ru.n_words, vocab_ru, device=device, max_length=MAX_LENGTH)
model = Transformer(encoder, decoder).to(device)

if start_epoch != 1: # Continue the existing training
    model = torch.load(model_path)

print(sum(p.numel() for p in model.parameters())/1e6, 'Million parameters')

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(ignore_index=vocab_ru.word2index['<pad>'])

loss_train, loss_val = train(
    model, 
    train_dataloader, 
    val_dataloader, 
    optimizer=optimizer, 
    criterion=criterion, 
    start_epoch=start_epoch,
    epochs=epochs, 
    lr=lr, 
    model_path=model_path
)

42.668816 Million parameters


Epoch 20: val: 100%|██████████| 146/146 [00:04<00:00, 30.49it/s, loss=3.07]


# Check in random sample from dataset

In [9]:
def evaluate(model, sentence, vocab_en, vocab_ru):
    with torch.no_grad():
        model.eval()
        input_tensor = tensorFromSentence(vocab_en, sentence, device=device)
        outputs = model.generate_beam(input_tensor, max_length=model.decoder.max_length)

        ids = outputs.squeeze()
        
        words = []
        for idx in ids:
            if idx.item() == vocab_ru.word2index['<eos>']:
                break
            words.append(vocab_ru.index2word[idx.item()])
    return words

In [10]:
import random
import string

def evaluateRandomly(model, vocab_tox, vocab_detox, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('origin:     ', pair[0])
        print('translated: ', pair[1])
        output_words = evaluate(model, pair[0], vocab_tox, vocab_detox)
        output_sentence = "".join([" "+i if not i.startswith("'") and not i.startswith("n'") and i not in string.punctuation else i for i in output_words]).strip()
        print('predicted:  ', output_sentence)
        print('')

In [11]:
load_model = torch.load(model_path)

load_model.eval()
evaluateRandomly(load_model, vocab_en, vocab_ru)

origin:      novosibirsk opera and ballet theatre is 7 km from grand hotel , while novosibirsk state circus is 8 km away .
translated:  отель грандотель находится в 7 км от новосибирского театра оперы и балета и в 8 км от новосибирского государственного цирка .
predicted:   <sos> в 7 км от отеля типа постель и завтрак театр оперы и балета, а расстояние до театра оперы составляет 8 км.

origin:      hotel terranova is located in lens zona piel area , mexicos leather capital .
translated:  отель terranova находится в районе леона под названием зона пьель , который в мексике считается столицей кожаных изделий .
predicted:   <sos> в отеле vienna для некурящих гостей отеля vienna, расположенном в семье районе авлабари, в отеле, в отеле, в отеле работает круглосуточная стойка доступности.

origin:      the owner can provide fresh bread in the morning .
translated:  по утрам владелец апартаментов может доставлять свежий хлеб .
predicted:   <sos> в здании отеля подают блюда по меню, который вк